In [ ]:
import numpy as np
from time import sleep
from matplotlib import pyplot as plt
from matplotlib import colors

%matplotlib inline

In [ ]:
class Results(object):
    def __init__(self, baseline_time, baseline_label, color):
        self.speedups = [1]
        self.labels = [baseline_label]
        self.colors = [color]
        self.baseline_time = baseline_time
        self.times = []
        
         # ugly code to handle Jupyter version differences
        try:
            # newer versions of TimeItResult contain the average time already
            self.times.append(self.baseline_time.average)
        except:
            # Failing that, older versions have a list of all times
            self.times.append(np.mean(self.baseline_time.all_runs))
        
    def speedup_plot(self, log_scale=False):
        ind = np.arange(len(self.speedups))
        fig, ax = plt.subplots(figsize=(12,8))
        
        for name, index, speedup, color in zip(self.labels, ind, self.speedups, self.colors):
            ax.bar(
                index,
                speedup, 
                width=0.5, 
                color=color,
                label=name,
                log=log_scale)
            
        ax.set_xticks(ind)
        ax.set_xticklabels(self.labels)
        ax.set_title('Speedup relative to {}'.format(self.labels[0]))
        ax.legend(self.labels)
        plt.show()
        return self
        
    def walltime_plot(self, log_scale=False):
        ind = np.arange(len(self.speedups))
        fig, ax = plt.subplots(figsize=(12,8))
        
        for name, index, walltime, color in zip(self.labels, ind, self.times, self.colors):
            ax.bar(
                index,
                walltime, 
                width=0.5, 
                color=color,
                label=name,
                log=log_scale)
            
        ax.set_xticks(ind)
        ax.set_xticklabels(self.labels)
        ax.set_title('Walltime')
        ax.legend(self.labels)
        plt.show()
        return self
        
    def add_time(self, time, label, color):
        
        baseline_avg = self.times[0]
        # ugly code to handle Jupyter version differences
        try:
            # newer versions of TimeItResult contain the average time already
            self.speedups.append(baseline_avg / time.average)
            self.times.append(time.average)
        except:
            # Failing that, older versions have a list of all times
            avg = np.mean(time.all_runs)
            self.speedups.append(baseline_avg / avg)
            self.times.append(avg)
            
        self.labels.append(label)
        self.colors.append(color)
        
        # return a reference to self to allow chaining
        return self

In [ ]:
times_baseline = %timeit -n 1 -r 1 -o results = sleep(0.5)

In [ ]:
results = Results(times_baseline, 'baseline', 'blue')

In [ ]:
t = %timeit -n 1 -r 1 -o results = sleep(0.2)
results.add_time(t, '1', 'green');

In [ ]:
t = %timeit -n 1 -r 1 -o results = sleep(0.1)
results.add_time(t, '2', 'yellow');

In [ ]:
results.speedup_plot();

In [ ]:
results.walltime_plot();